In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-ocr-data/preprocessed_ocr_train_data.csv
/kaggle/input/preprocessed-ocr-data/preprocessed_ocr_test_data.csv
/kaggle/input/final-data-btp/Final_train_data.csv
/kaggle/input/final-data-btp/Final_test_data.csv
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/config.json
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/trainer_state.json
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/training_args.bin
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/scheduler.pt
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/model.safetensors
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/optimizer.pt
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/rng_state.pth
/kaggle/input/btp_2epoch_checkpoints/transformers/default/1/generation_config.json


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import logging

# Set up logging
logging.basicConfig(
    filename="training_log.log",
    level=logging.INFO,
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger()


In [4]:
class EntityValueDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels["input_ids"][idx])
        return item

    def __len__(self):
        return len(self.labels["input_ids"])

class EntityValueDatasetTest(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(next(iter(self.encodings.values())))

def prepare_data(df):
    inputs = []
    targets = []
    for _, row in df.iterrows():
        input_text = f"Group ID: {row['group_id']}, Entity Name: {row['entity_name']}, Text: {row['cleaned_extracted_values']}"
        inputs.append(f"Extract {row['entity_name']} from: {input_text}")
        targets.append(str(row["entity_value"]))
    return inputs, targets


In [5]:
def calculate_f1_score(ground_truths, predictions):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for gt, pred in zip(ground_truths, predictions):
        if pred != "" and gt != "" and pred == gt:
            true_positives += 1
        elif pred != "" and gt != "" and pred != gt:
            false_positives += 1
        elif pred != "" and gt == "":
            false_positives += 1
        elif pred == "" and gt != "":
            false_negatives += 1

    precision = (
        true_positives / (true_positives + false_positives)
        if (true_positives + false_positives) > 0
        else 0
    )
    recall = (
        true_positives / (true_positives + false_negatives)
        if (true_positives + false_negatives) > 0
        else 0
    )
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1_score


In [6]:
def batch_predict(model, tokenizer, dataset, batch_size=16):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    predictions = []
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            outputs = model.generate(**inputs, max_length=32, do_sample=False)
            predictions.extend([tokenizer.decode(output, skip_special_tokens=True) for output in outputs])
    return predictions


train_df = pd.read_csv("/kaggle/input/final-data-btp/Final_train_data.csv")
test_df = pd.read_csv("/kaggle/input/final-data-btp/Final_test_data.csv")

# Filter the training dataset
pattern = r'^\d+(\.\d+)?\s+[a-zA-Z]+$'
train_df = train_df[train_df['entity_value'].str.match(pattern, na=False)]
train_df = train_df[~train_df['entity_value'].str.contains('horsepower', case=False, na=False)]

train_inputs, train_targets = prepare_data(train_df)
test_inputs, test_targets = prepare_data(test_df)

train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(train_inputs, train_targets, test_size=0.1, random_state=42)

In [7]:

model_name = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize data
train_encodings = tokenizer(train_inputs, truncation=True, padding=True, max_length=300)
train_target_encodings = tokenizer(train_targets, truncation=True, padding=True, max_length=32)

eval_encodings = tokenizer(eval_inputs, truncation=True, padding=True, max_length=300)
eval_target_encodings = tokenizer(eval_targets, truncation=True, padding=True, max_length=32)

test_encodings = tokenizer(test_inputs, truncation=True, padding=True, max_length=300)

# Create datasets
train_dataset = EntityValueDataset(train_encodings, train_target_encodings)
eval_dataset = EntityValueDataset(eval_encodings, eval_target_encodings)
test_dataset = EntityValueDatasetTest(test_encodings)



config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=1e-3,
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


trainer.train()


model.save_pretrained("./fine_tuned_t5_entity_value")
tokenizer.save_pretrained("./fine_tuned_t5_entity_value")

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.133100,0.147029
2,0.097700,0.125532


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('./fine_tuned_t5_entity_value/tokenizer_config.json',
 './fine_tuned_t5_entity_value/special_tokens_map.json',
 './fine_tuned_t5_entity_value/spiece.model',
 './fine_tuned_t5_entity_value/added_tokens.json',
 './fine_tuned_t5_entity_value/tokenizer.json')

In [10]:

model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_t5_entity_value")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [11]:
from tqdm import tqdm
from torch.utils.data import DataLoader

def batch_predict(batch_group_ids, batch_entity_names, batch_texts):
    input_texts = [
        f"Extract {entity_name} from: Group ID: {group_id}, Entity Name: {entity_name}, Text: {text}"
        for group_id, entity_name, text in zip(batch_group_ids, batch_entity_names, batch_texts)
    ]
    inputs = tokenizer(input_texts, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=32, num_return_sequences=1, do_sample=False)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

class TestDataset(Dataset):
    def __init__(self, df):
        self.group_ids = df['group_id'].tolist()
        self.entity_names = df['entity_name'].tolist()
        self.texts = df['cleaned_extracted_values'].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.group_ids[idx], self.entity_names[idx], self.texts[idx]

test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predicted_values = []
print("Starting prediction...")
for batch in tqdm(test_loader, desc="Predicting", unit="batch"):
    batch_group_ids, batch_entity_names, batch_texts = batch
    batch_predictions = batch_predict(batch_group_ids, batch_entity_names, batch_texts)
    predicted_values.extend(batch_predictions)

test_df['predicted_value'] = predicted_values

f1_score = calculate_f1_score(test_df['entity_value'], test_df['predicted_value'])
print(f"F1 Score on Test Data: {f1_score}")

Starting prediction...


Predicting: 100%|██████████| 890/890 [08:57<00:00,  1.65batch/s]

F1 Score on Test Data: 0.7531961471103328


In [12]:
test_df.to_csv("test_data_with_prediction.csv")

In [19]:
import torch

# Clear memory and reset
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()  # Resets the peak memory tracker
torch.cuda.reset_accumulated_memory_stats()  # Resets accumulated memory tracker


In [8]:
model_dir = "/kaggle/input/btp_2epoch_checkpoints/transformers/default/1"

output_dir = "/kaggle/working/"
if os.path.exists(model_dir):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
    print("Resuming training from saved model.")
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    print("Initializing new model.")

Resuming training from saved model.


In [9]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define training arguments
resume_training = os.path.exists(model_dir)
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,  # Specify additional epochs
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=1e-3,
    resume_from_checkpoint=resume_training, 
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.110600,0.139508
2,0.087800,0.122091


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=12766, training_loss=0.10309239999732123, metrics={'train_runtime': 14227.5444, 'train_samples_per_second': 26.914, 'train_steps_per_second': 0.897, 'total_flos': 1.36630882833408e+17, 'train_loss': 0.10309239999732123, 'epoch': 2.0})

In [10]:
model_dir = "./kaggle/working/fine_tuned_t4_entity_value"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)
print(f"Model and tokenizer saved to {model_dir}")

Model and tokenizer saved to ./kaggle/working/fine_tuned_t4_entity_value


In [12]:
test_df = pd.read_csv("/kaggle/input/final-data-btp/Final_test_data.csv")

In [12]:
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader

def batch_predict(batch_group_ids, batch_entity_names, batch_texts):
    input_texts = [
        f"Extract {entity_name} from: Group ID: {group_id}, Entity Name: {entity_name}, Text: {text}"
        for group_id, entity_name, text in zip(batch_group_ids, batch_entity_names, batch_texts)
    ]
    inputs = tokenizer(input_texts, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=32, num_return_sequences=1, do_sample=False)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


# Prepare DataLoader for test data
class TestDataset(Dataset):
    def __init__(self, df):
        self.group_ids = df['group_id'].tolist()
        self.entity_names = df['entity_name'].tolist()
        self.texts = df['cleaned_extracted_values'].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.group_ids[idx], self.entity_names[idx], self.texts[idx]


test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Perform batch prediction and track progress
predicted_values = []
print("Starting prediction...")
for batch in tqdm(test_loader, desc="Predicting", unit="batch"):
    batch_group_ids, batch_entity_names, batch_texts = batch
    batch_predictions = batch_predict(batch_group_ids, batch_entity_names, batch_texts)
    predicted_values.extend(batch_predictions)

test_df['predicted_value'] = predicted_values

f1_score = calculate_f1_score(test_df['entity_value'], test_df['predicted_value'])
print(f"F1 Score on Test Data: {f1_score}")


Starting prediction...


Predicting: 100%|██████████| 890/890 [08:55<00:00,  1.66batch/s]

F1 Score on Test Data: 0.7711129331549649


In [15]:
model_dir = "/kaggle/working/checkpoint-12766"

output_dir = "/kaggle/working/"
if os.path.exists(model_dir):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
    print("Resuming training from saved model.")
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    print("Initializing new model.")

Resuming training from saved model.


In [17]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define training arguments
resume_training = os.path.exists(model_dir)
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1, 
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=1e-3,
    resume_from_checkpoint=resume_training, 
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.075000,0.136780


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=6383, training_loss=0.07449931010881711, metrics={'train_runtime': 7113.3723, 'train_samples_per_second': 26.916, 'train_steps_per_second': 0.897, 'total_flos': 6.8315441416704e+16, 'train_loss': 0.07449931010881711, 'epoch': 1.0})

In [18]:
model_dir = "./kaggle/working/fine_tuned_t4_t5thepoch_entity_value"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)
print(f"Model and tokenizer saved to {model_dir}")

Model and tokenizer saved to ./kaggle/working/fine_tuned_t4_t5thepoch_entity_value


In [19]:
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader

def batch_predict(batch_group_ids, batch_entity_names, batch_texts):
    input_texts = [
        f"Extract {entity_name} from: Group ID: {group_id}, Entity Name: {entity_name}, Text: {text}"
        for group_id, entity_name, text in zip(batch_group_ids, batch_entity_names, batch_texts)
    ]
    inputs = tokenizer(input_texts, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=32, num_return_sequences=1, do_sample=False)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


# Prepare DataLoader for test data
class TestDataset(Dataset):
    def __init__(self, df):
        self.group_ids = df['group_id'].tolist()
        self.entity_names = df['entity_name'].tolist()
        self.texts = df['cleaned_extracted_values'].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.group_ids[idx], self.entity_names[idx], self.texts[idx]


test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Perform batch prediction and track progress
predicted_values = []
print("Starting prediction...")
for batch in tqdm(test_loader, desc="Predicting", unit="batch"):
    batch_group_ids, batch_entity_names, batch_texts = batch
    batch_predictions = batch_predict(batch_group_ids, batch_entity_names, batch_texts)
    predicted_values.extend(batch_predictions)

test_df['predicted_value'] = predicted_values

f1_score = calculate_f1_score(test_df['entity_value'], test_df['predicted_value'])
print(f"F1 Score on Test Data: {f1_score}")


Starting prediction...


Predicting: 100%|██████████| 890/890 [08:55<00:00,  1.66batch/s]

F1 Score on Test Data: 0.7685089084933402


In [21]:
import shutil
model_dir = "/kaggle/working/kaggle/working/fine_tuned_t4_t5thepoch_entity_value"
# Zip the model directory
shutil.make_archive("fine_tuned_t5_5epoch", 'zip', model_dir)

# Download the zip file
from IPython.display import FileLink
FileLink("fine_tuned_t5_5epoch.zip")


/kaggle/working/fine_tuned_t5_5epoch.zip

In [14]:
import shutil
model_dir1 = "/kaggle/working/checkpoint-12766"

shutil.make_archive("fine_tuned_t5_checpoint_epoch4", 'zip', model_dir1)

# Download the zip file
from IPython.display import FileLink
FileLink("fine_tuned_t5_checpoint_epoch4.zip")

/kaggle/working/fine_tuned_t5_checpoint_epoch4.zip

In [ ]:
import time
time.sleep(10000)